In [24]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import io
import time
import random
import seaborn # Attention 시각화를 위해 필요!
import sentencepiece as spm

## 데이터 정제 및 토큰화

In [2]:
data_dir = os.getenv('HOME')+'/aiffel/transformer/data'
kor_path = data_dir+"/korean-english-park.train.ko"
eng_path = data_dir+"/korean-english-park.train.en"

# 데이터 정제 및 토큰화
def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r") as f: kor = f.read().splitlines()
    with open(eng_path, "r") as f: eng = f.read().splitlines()
    assert len(kor) == len(eng)

    # 병렬 쌍을 집합(set)으로 만들어 중복 제거
    unique_pairs = set(zip(kor, eng))
    
    # 리스트로 변환하여 다시 분리
    cleaned_corpus = list(unique_pairs)

    return cleaned_corpus

cleaned_corpus = clean_corpus(kor_path, eng_path)

In [3]:
def preprocess_sentence(sentence):
    
     # 1. 모든 입력을 소문자로 변환
    sentence = sentence.lower()
    
    # 2. 알파벳, 문장부호, 한글만 남기고 모두 제거
    sentence = re.sub(r"[^a-zA-Z가-힣.,!?]", " ", sentence)
    
    # 3. 문장부호 양옆에 공백을 추가
    sentence = re.sub(r"([.,!?])", r" \1 ", sentence)
    
    # 4. 문장 앞뒤의 불필요한 공백을 제거
    sentence = sentence.strip()
    
    return sentence

In [4]:
# Sentencepiece를 활용하여 학습한 tokenizer를 생성합니다.
def generate_tokenizer(corpus,
                        vocab_size,
                        lang="ko",
                        pad_id=0,
                        bos_id=1,
                        eos_id=2,
                        unk_id=3):
    # 모델 파일 경로 설정
    model_prefix = f"{lang}_tokenizer"
    input_file = f"{lang}_corpus.txt"

    # 코퍼스를 텍스트 파일로 저장 (SentencePiece는 파일 형태 입력 필요)
    with open(input_file, 'w') as f:
        for sentence in corpus:
            f.write(f"{sentence}\n")

    # SentencePiece 모델 학습
    spm.SentencePieceTrainer.Train(
        f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
        f"--pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}"
    )

    # 학습된 모델을 로드하여 SentencePieceProcessor 반환
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f"{model_prefix}.model")

    return tokenizer
    

SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

eng_corpus = []
kor_corpus = []

for k, e in cleaned_corpus:
    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=ko_corpus.txt --model_prefix=ko_tokenizer --vocab_size=20000 --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ko_corpus.txt
  input_format: 
  model_prefix: ko_tokenizer
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <un

True

177) LOG(INFO) Running command: --input=en_corpus.txt --model_prefix=en_tokenizer --vocab_size=20000 --pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: en_corpus.txt
  input_format: 
  model_prefix: en_tokenizer
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos

In [5]:
from tqdm.notebook import tqdm    # Process 과정을 보기 위해

src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰의 길이가 50 이하인 문장만 남깁니다. 
for idx in tqdm(range(len(kor_corpus))):
    # 한국어와 영어 문장을 각각 토큰화
    kor_tokens = ko_tokenizer.EncodeAsIds(kor_corpus[idx])
    eng_tokens = en_tokenizer.EncodeAsIds(eng_corpus[idx])
    
    # 토큰의 길이가 50 이하인 경우에만 추가
    if len(kor_tokens) <= 50 and len(eng_tokens) <= 50:
        src_corpus.append(kor_tokens)
        tgt_corpus.append(eng_tokens)

# 패딩처리를 완료하여 학습용 데이터를 완성합니다. 
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

  0%|          | 0/78968 [00:00<?, ?it/s]

## 모델 설계

In [8]:
#1. Positional Encoding
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    return sinusoid_table

In [9]:
#2. Multi-Head Attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
            
        self.depth = d_model // self.num_heads
            
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
            
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
            

    def split_heads(self, x):
        batch_size = x.shape[0]
        split_x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        batch_size = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

        return combined_x

        
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
            
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
            
        out = self.combine_heads(out)
        out = self.linear(out)
                
        return out, attention_weights

In [11]:
#3. Position-wise Feed-Forward Network
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
            
        return out

In [12]:
#4. Encoder 레이어 구현하기
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual
        
        return out, enc_attn

In [14]:
#5. Decoder 레이어 구현하기
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.dropout(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
        out = self.dropout(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [15]:
#6. Encoder, Decoder 클래스 정의하기
class Encoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [16]:
class Decoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [35]:
#7. Transformer 완성하기
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared=True):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)

        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared = shared

        if shared: self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]
        out = emb(x)

        if self.shared: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.dropout(out)

        return out
        
    def call(self, enc_in, dec_in):
        enc_mask, dec_enc_mask, dec_mask = generate_masks(enc_in, dec_in)
        
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, dec_mask, dec_enc_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

    def train_step(self, data):
        # 데이터 언패킹
        enc_in, dec_in, y_true = data
        
        # 예측 및 손실 계산
        with tf.GradientTape() as tape:
            y_pred, _, _, _ = self(enc_in, dec_in)
            loss = self.compiled_loss(y_true, y_pred)

        # 기울기 계산 및 적용
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # 지표 업데이트
        self.compiled_metrics.update_state(y_true, y_pred)
        return {m.name: m.result() for m in self.metrics}


In [40]:
# mask
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_causality_mask(tf.shape(tgt)[1])  # 시퀀스 길이를 단일 정수로 전달
    dec_enc_mask = generate_padding_mask(src)  # 인코더-디코더 마스크는 소스 시퀀스에 대해 생성

    return enc_mask, dec_enc_mask, dec_mask

In [41]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98, 
                                     epsilon=1e-9)

## 훈련하기

In [42]:
# 하이퍼파라미터 설정
n_layers = 2
d_model = 512
n_heads = 8
d_ff = 2048
dropout = 0.1
src_vocab_size = SRC_VOCAB_SIZE
tgt_vocab_size = TGT_VOCAB_SIZE
pos_len = 1000

# Transformer 모델 초기화
transformer = Transformer(
    n_layers=n_layers,
    d_model=d_model,
    n_heads=n_heads,
    d_ff=d_ff,
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    pos_len=pos_len,
    dropout=dropout
)

# 옵티마이저 및 손실 함수
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 학습 루프 정의
@tf.function
def train_step(enc_in, dec_in, y_true):
    with tf.GradientTape() as tape:
        # 모델 출력
        y_pred, _, _, _ = transformer(enc_in, dec_in)
        
        # 손실 계산
        loss = loss_object(y_true, y_pred)

    # 기울기 계산 및 적용
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    
    return loss

# 예시 학습 데이터
enc_input_example = np.random.randint(0, src_vocab_size, (16, 50))
dec_input_example = np.random.randint(0, tgt_vocab_size, (16, 50))
y_true_example = dec_input_example

# 학습 실행
EPOCHS = 1
for epoch in range(EPOCHS):
    loss = train_step(enc_input_example, dec_input_example, y_true_example)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

Epoch 1, Loss: 10.0724


In [44]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [50]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 20

examples = [
            "오바마는 대통령이다.",
            "시민들은 도시 속에 산다.",
            "커피는 필요 없다.",
            "일곱 명의 사망자가 발생했다."
]

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

for (batch, idx) in enumerate(t):
    batch_loss = train_step(
        enc_train[idx:idx+BATCH_SIZE],
        dec_train[idx:idx+BATCH_SIZE],
        dec_train[idx:idx+BATCH_SIZE]
    )

    total_loss += batch_loss
    t.set_description_str('Epoch %2d' % (epoch + 1))
    t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))
    
def translate(sentence, transformer, ko_tokenizer, en_tokenizer):
    # 입력 문장을 토큰화
    sentence_tokens = ko_tokenizer.EncodeAsIds(sentence)
    sentence_tensor = tf.convert_to_tensor([sentence_tokens])

    # 초기 디코더 입력 준비 (<BOS> 토큰)
    decoder_input = [en_tokenizer.bos_id()]
    output = tf.convert_to_tensor([decoder_input])

    # 디코더를 통해 단어를 하나씩 생성
    for _ in range(50):  # 최대 50개의 토큰 생성
        enc_mask, dec_enc_mask, dec_mask = generate_masks(sentence_tensor, output)
        predictions, _, _, _ = transformer(sentence_tensor, output, enc_mask, dec_mask, dec_enc_mask)

        # 마지막 단어 예측
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]

        # <EOS> 토큰이 나오면 중단
        if predicted_id == en_tokenizer.eos_id():
            break

        # 예측 단어를 출력에 추가
        output = tf.concat([output, tf.convert_to_tensor([[predicted_id]])], axis=-1)

    # 토큰을 텍스트로 변환하여 출력
    translated_text = en_tokenizer.DecodeIds(output[0].numpy().tolist())
    print(f"Input: {sentence}")
    print(f"Translated: {translated_text}")

    for example in examples:
        translate(example, transformer, ko_tokenizer, en_tokenizer)

/tmp/ipykernel_427/1080583397.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm_notebook(idx_list)


  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

...

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

  0%|          | 0/1128 [00:00<?, ?it/s]

In [51]:
def translate(sentence, transformer, ko_tokenizer, en_tokenizer):
    # 입력 문장을 토큰화
    sentence_tokens = ko_tokenizer.EncodeAsIds(sentence)
    print(f"Tokenized Input Sentence: {sentence_tokens}")  # 디버그용 출력
    sentence_tensor = tf.convert_to_tensor([sentence_tokens])

    # 초기 디코더 입력 준비 (<BOS> 토큰)
    decoder_input = [en_tokenizer.bos_id()]
    output = tf.convert_to_tensor([decoder_input])

    # 디코더를 통해 단어를 하나씩 생성
    for _ in range(50):  # 최대 50개의 토큰 생성
        enc_mask, dec_enc_mask, dec_mask = generate_masks(sentence_tensor, output)
        predictions, _, _, _ = transformer(sentence_tensor, output, enc_mask, dec_mask, dec_enc_mask)

        # 마지막 단어 예측
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]
        print(f"Predicted ID: {predicted_id}")  # 디버그용 출력

        # <EOS> 토큰이 나오면 중단
        if predicted_id == en_tokenizer.eos_id():
            break

        # 예측 단어를 출력에 추가
        output = tf.concat([output, tf.convert_to_tensor([[predicted_id]])], axis=-1)

    # 토큰을 텍스트로 변환하여 출력
    translated_text = en_tokenizer.DecodeIds(output[0].numpy().tolist())
    print(f"Input: {sentence}")
    print(f"Translated: {translated_text}")

## 회고

에포크만 돌아가고 번역 결과는 출력이 안됨, 왜그럴까...
중간에 오류가 많이 생겨서 디버깅하느라 시간 꽤 소요됨